GPU check

In [1]:
!nvidia-smi

Wed Aug 27 15:51:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 561.19                 Driver Version: 561.19         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   41C    P3             11W /   30W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [25]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, default_data_collator
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [26]:
model_path = "gemma_model" 
tokenizer_path = "gemma_tokenizer"  
dataset_path = "A:/gemma270-rag/data/100_sample_query_finetuning.csv"
output_dir = "outputs"

In [27]:
q_lora = False  # True -> QLoRA, False -> LoRA
lora_r, lora_alpha, lora_dropout = 8, 16, 0.05
brevity_B, brevity_lambda = 10, 0.1


In [28]:
from datasets import load_dataset

dataset_path = "A:/gemma270-rag/data/100_sample_query_finetuning.csv"

# load csv
ds = load_dataset("csv", data_files=dataset_path)

# pick train split
dataset = ds["train"]

# drop unwanted columns (like id, source)
dataset = dataset.remove_columns([col for col in dataset.column_names if col not in ["query", "answer", "theme"]])


def build_prompt(ex):
    prompt = f"Query: {ex['query']}\nAnswer:"
    full = f"Query: {ex['query']}\nAnswer: {ex['answer']}"
    return prompt, full
def preprocess(ex):
    prompt, full = build_prompt(ex)
    prompt_ids = tokenizer(prompt, truncation=True, max_length=512)["input_ids"]
    full_enc = tokenizer(full, truncation=True, max_length=512)
    labels = full_enc["input_ids"].copy()
    for i in range(len(prompt_ids)):
        if i < len(labels):
            labels[i] = -100
    full_enc["labels"] = labels
    return full_enc

In [29]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({"pad_token": "<pad>"})

In [30]:
if q_lora:
    from transformers import BitsAndBytesConfig
    bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)
    model = AutoModelForCausalLM.from_pretrained(model_path, quantization_config=bnb_config, device_map="auto")
    model = prepare_model_for_kbit_training(model)
else:
    model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16, device_map="auto")

In [31]:
model.resize_token_embeddings(len(tokenizer))
tokenized_ds = ds["train"].map(
    preprocess,
    remove_columns=ds["train"].column_names
)

target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "up_proj", "down_proj"]
config = LoraConfig(r=lora_r, lora_alpha=lora_alpha, target_modules=target_modules, lora_dropout=lora_dropout, task_type="CAUSAL_LM")
model = get_peft_model(model, config)

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(**inputs)
        loss = outputs.loss
        labels = inputs["labels"]
        valid_len = (labels != -100).sum(dim=-1).float()
        bp = torch.mean(torch.clamp(brevity_B - valid_len, min=0) / brevity_B)
        loss = loss + brevity_lambda * bp
        return (loss, outputs) if return_outputs else loss

args = TrainingArguments(output_dir=output_dir, num_train_epochs=3, per_device_train_batch_size=8, gradient_accumulation_steps=2, learning_rate=2e-4, bf16=True, save_strategy="epoch", logging_steps=50, remove_unused_columns=False)

trainer = CustomTrainer(model=model, args=args, train_dataset=tokenized_ds, tokenizer=tokenizer, data_collator=default_data_collator)

trainer.train()
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


Map: 100%|██████████| 100/100 [00:00<00:00, 1421.64 examples/s]


ValueError: Your setup doesn't support bf16/gpu.